In [1]:
import sys
sys.path.append('../')

# ต้อง ลงทะเบียน dataset ทุกครั้ง ที่ยังไม่ได้ลงทะเบียน เพื่อนำไปเทรน :D
from detectron2.data.datasets import register_coco_instances
register_coco_instances("tomato_train", {}, "C:/dev/dataset/tomato_mrcnn/laboro_tomato/annotations/train.json", "C:/dev/dataset/tomato_mrcnn/laboro_tomato/train")
register_coco_instances("tomato_test", {}, "C:/dev/dataset/tomato_mrcnn/laboro_tomato/annotations/test.json", "C:/dev/dataset/tomato_mrcnn/laboro_tomato/test")

import os
import time
import logging
import numpy as np
import cv2
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultTrainer
from detectron2.modeling import build_model
import datetime

logger = logging.getLogger()

class ImageProcessingMapper(DatasetMapper):
    def __init__(self, cfg, is_train=True, kernel_size=5, gamma=1.0, blur_enable=False):
        super(ImageProcessingMapper, self).__init__(cfg, is_train)
        self.kernel_size = kernel_size
        self.gamma = gamma
        self.blur_enable = blur_enable

    def apply_image_processing(self, image):
        if self.blur_enable:
            image = cv2.GaussianBlur(image, (self.kernel_size, self.kernel_size), 0)

        lookUpTable = np.empty((1, 256), np.uint8)
        for i in range(256):
            lookUpTable[0, i] = np.clip(pow(i / 255.0, self.gamma) * 255.0, 0, 255)
        return cv2.LUT(image.astype(np.uint8), lookUpTable)

    def __call__(self, dataset_dict):
        dataset_dict = super(ImageProcessingMapper, self).__call__(dataset_dict)
        image = dataset_dict["image"].numpy().transpose(1, 2, 0)
        image = self.apply_image_processing(image)
        dataset_dict["image"] = torch.Tensor(image.transpose(2, 0, 1))
        return dataset_dict

def log_model_details(model, logger):
    param_details = []
    for name, param in model.named_parameters():
        param_details.append(f"{name}: {param.size()}")

    total_params = sum(p.numel() for p in model.parameters())
    param_details.append(f'Total number of parameters: {total_params}')

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    param_details.append(f'Trainable parameters: {trainable_params}')
    param_details.append(f'Non-trainable parameters: {non_trainable_params}')

    for detail in param_details:
        logger.info(detail)


def setup_directories(model_used):
    base_output_dir = os.path.join(".", "train", model_used)
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    specific_output_dir = os.path.join(base_output_dir, current_time)

    before_test_dir = os.path.join(specific_output_dir, "before")
    after_test_dir = os.path.join(specific_output_dir, "after")
    os.makedirs(before_test_dir, exist_ok=True)
    os.makedirs(after_test_dir, exist_ok=True)

    log_dir = os.path.join(specific_output_dir)
    os.makedirs(log_dir, exist_ok=True)

    log_file_path = setup_logging(log_dir)
    return before_test_dir, after_test_dir, log_file_path


def setup_logging(log_dir, log_filename="training.log"):
    log_file_path = os.path.join(log_dir, log_filename)
    logger = logging.getLogger()
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)

    file_handler = logging.FileHandler(log_file_path)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.setLevel(logging.INFO)

    return log_file_path


def configure_model(model_input, lr_, epoch_, input_size_, dv_):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(model_input))
    cfg.DATASETS.TRAIN = ("tomato_train",)
    cfg.DATASETS.TEST = ("tomato_test",)
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_input)
    cfg.SOLVER.IMS_PER_BATCH = 1
    cfg.SOLVER.BASE_LR = lr_
    cfg.SOLVER.MAX_ITER = epoch_
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
    cfg.INPUT.MIN_SIZE_TRAIN = input_size_
    cfg.MODEL.DEVICE = dv_
    return cfg


def save_config(cfg, model_base, model_used):
    config_directory = f'./train/config/{model_base}/'
    os.makedirs(config_directory, exist_ok=True)
    config_path = os.path.join(config_directory, model_used)
    with open(config_path, 'w') as f:
        f.write(cfg.dump())


def save_model_summary(model, model_base, model_used):
    model_summary_directory = f'./train/model_summary/{model_base}/'
    os.makedirs(model_summary_directory, exist_ok=True)
    model_summary_path = os.path.join(model_summary_directory, model_used.replace('.yaml', '.txt'))
    with open(model_summary_path, 'w') as f:
        print(str(model), file=f)


class Trainer(DefaultTrainer):
    before_test_dir = None

    def __init__(self, cfg, before_test_dir=None):
        super(Trainer, self).__init__(cfg)
        Trainer.before_test_dir = before_test_dir

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = cls.before_test_dir
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

def train_model(cfg, before_test_dir):
    """
    Train the model using the given configuration.
    """
    trainer = Trainer(cfg, before_test_dir=before_test_dir)
    trainer.resume_or_load(resume=False)
    trainer.train()

def main():
    model_base = "COCO-InstanceSegmentation"
    model_used = "mask_rcnn_R_50_C4_1x.yaml"
    epoch_ = 250
    lr_ = 0.001
    input_size_ = (336, 336)
    dv_ = "cuda"
    setup_step_ = 0
    model_input = f"{model_base}/{model_used}"
    blur_enable = False
    blur_kernel_size = 3
    gamma_correction = 1.0

    before_test_dir, after_test_dir, log_file_path = setup_directories(model_used)
    logger = logging.getLogger()
    cfg = configure_model(model_input, lr_, epoch_, input_size_, dv_)
    save_config(cfg, model_base, model_used)

    model = build_model(cfg)
    save_model_summary(model, model_base, model_used)
    log_model_details(model, logger)
    #START TRAIN BRO!!
    start_time = time.time()
    train_model(cfg, before_test_dir)
    end_time = time.time()

    total_time = end_time - start_time
    minutes = total_time // 60
    seconds = total_time % 60

    logger.info(f"บันทึกโมเดลทดสอบเสร็จแล้วที่ : {before_test_dir}")
    logger.info(f"Base Model: {model_input}")
    logger.info(f"จำนวณรอบการฝึกอบรม (epoch): {epoch_}")
    logger.info(f"จำนวณอัตราการเรียนรู้ (lr)  : {lr_}")
    logger.info(f"ขนาดต้นทาง (input size) : {input_size_}")
    logger.info(f"อุปกรณ์ที่ใช้ฝึกสอน (device): {dv_}")
    logger.info(f"เวลาที่ใช้ในการฝึกสอน: {minutes} นาที {seconds:.2f} วินาที")


if __name__ == '__main__':
    torch.cuda.empty_cache()
    main()
    torch.cuda.empty_cache()
    # ... (Other code)

ImportError: cannot import name 'Hook' from 'detectron2.engine' (c:\dev\dataset\tomato_mrcnn\detectron2\engine\__init__.py)